In [133]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [134]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=0, bias=0),     #Input-1x28x28      Output-16x26x26    RF-3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 32, 3, padding=0, bias=0),    #Input-16x26x26    Output-32x24x24    RF-5
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32, 16, 1),                       #Input-32x24x24    Output-16x24x24    RF-10
            nn.MaxPool2d(2, 2)                          #Input-16x24x24    Output-16x12x12    RF-10
            
            
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=0, bias=0),    #Input-16x12x12    Output-16x10x10    RF-12
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.1),
            nn.Conv2d(16, 32, 3, padding=0, bias=0),    #Input-16x10x10    Output-32x8x8      RF-14
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32, 16, 1),                       #Input-32x8x8      Output-16x8x8    RF-28
            nn.MaxPool2d(2, 2)                          #Input-16x4x4      Output-16x4x4    RF-28 
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1, bias=0),    #Input-16x4x4    Output-32x4x4    RF-30
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),

            nn.Conv2d(32, 56, 1)                        #Input-32x4x4    Output-56x4x4    RF-30 
        )
        
        self.avgpool = nn.AvgPool2d(4)                  #Input-56x4x4    Output-56x1x1
        self.out = nn.Conv2d(56, 10, 1, bias=0)         #Input-56x1x1    Output-10x1x1        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.avgpool(x)
        x = self.out(x)
        x = x.squeeze()
        x = F.log_softmax(x, dim=1)
        return x


In [135]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

You should consider upgrading via the '/home/sahil/miniconda3/envs/deeplearning/bin/python -m pip install --upgrade pip' command.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,608
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 16, 24, 24]             528
        MaxPool2d-10           [-1, 16, 12, 12]               0
           Conv2d-11           [-1, 16, 10, 10]           2,304
             ReLU-12           [-1, 1

In [136]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [137]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [138]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0766, Accuracy: 9784/10000 (97.84%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0452, Accuracy: 9855/10000 (98.55%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0388, Accuracy: 9881/10000 (98.81%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0352, Accuracy: 9889/10000 (98.89%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0295, Accuracy: 9907/10000 (99.07%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0263, Accuracy: 9925/10000 (99.25%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0280, Accuracy: 9910/10000 (99.10%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0303, Accuracy: 9904/10000 (99.04%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0242, Accuracy: 9929/10000 (99.29%)

  0%|          | 0/469 [00:00<?, ?it/s]
Test set: Average loss: 0.0301, A